# Qualidade de vinhos

Neste notebook, buscamos criar um modelo de regressão linear que avalie a qualidade dos vinhos a partir de suas propriedades.

Para isso, primeiramente vamos carregar os dados que temos dos vinhos e da qualidade destes, e depois vamos escalar os dados usando o escalador padrão, que mapeia os dados para uma distribuição normal:

In [89]:
import pandas as pd
from sklearn import preprocessing

winewhite = pd.read_csv('winequality-white.csv', sep=';')
wineScaled = winewhite.copy()

scaler = preprocessing.StandardScaler()
wineScaled.iloc[:, 0:11] = scaler.fit_transform(wineScaled.iloc[:, 0:11])

print('Data:')
display(wineScaled.head(3))


Data:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.172097,-0.081770,0.213280,2.821349,-0.035355,0.569932,0.744565,2.331512,-1.246921,-0.349184,-1.393152,6
1,-0.657501,0.215896,0.048001,-0.944765,0.147747,-1.253019,-0.149685,-0.009154,0.740029,0.001342,-0.824276,6
2,1.475751,0.017452,0.543838,0.100282,0.193523,-0.312141,-0.973336,0.358665,0.475102,-0.436816,-0.336667,6


Após isso, podemos separar os dados e valores de qualidade em conjuntos de treino e teste:

In [90]:
from sklearn.model_selection import train_test_split

(trainData, testData, trainLabels, testLabels) = train_test_split(wineScaled.iloc[:, 0:11], wineScaled['quality'],
                                                                test_size=0.2, random_state=42)
print('Train data and labels:')
display(trainData.head(3))
display(trainLabels.head(3))


Train data and labels:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
4665,0.527639,-1.073988,0.213280,0.356615,-0.813539,0.511127,-0.643875,-0.437162,-0.319678,-0.699710,1.532500
1943,-0.657501,-0.280214,0.874397,1.027022,-0.218457,0.746346,1.332887,0.927112,-0.054751,0.264236,-0.824276
3399,-1.487099,0.414339,-0.034638,0.198872,-0.401559,-0.606165,-1.020401,-0.450537,0.408870,0.001342,0.476014


4665    6
1943    5
3399    6
Name: quality, dtype: int64

Agora que temos os dados, podemos treinar um modelo de regressão linear com gradiente descendente para prever as as qualidades dos vinhos.

Após isso, vamos prever as qualidades para o treino e teste e calcular os erros médios absolutos e quadráticos para cada um dos conjuntos:

In [91]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

sqrRegressor = SGDRegressor()
sqrRegressor.fit(trainData, trainLabels)

sqrTrainPred = sqrRegressor.predict(trainData)
sqrTestPred = sqrRegressor.predict(testData)

maeTrain = mean_absolute_error(sqrTrainPred, trainLabels)
mseTrain = mean_squared_error(sqrTrainPred, trainLabels)
maeTest = mean_absolute_error(sqrTestPred, testLabels)
mseTest = mean_squared_error(sqrTestPred, testLabels)
print(f'Mean absolute error for train data: {maeTrain:.2f}')
print(f'Mean squared error for train data: {mseTrain:.2f}')
print(f'Mean absolute error for test data: {maeTest:.2f}')
print(f'Mean squared error for test data: {mseTest:.2f}')


Mean absolute error for train data: 0.58
Mean squared error for train data: 0.57
Mean absolute error for test data: 0.59
Mean squared error for test data: 0.58


É notável (mesmo que bem pequena) a diferença entre os erros para os grupos de treino e teste, sendo menor para o conjunto de treino, visto que o modelo foi treinado para minimizar o erro nesse conjunto.

Também percebemos que os erros quadráticos são menores que os absolutos. Neste caso, isso ocorre porque quadrados de valores abaixo de zero são menores que os próprios valores, e, como a média dos erros é abaixo de zero, é provável que a maioria dos erros foi, também, abaixo de zero.